In [1]:
# import set 
using Turing
using Distances
using LinearAlgebra
using LimberJack
using Interpolations
using GaussianProcess
using Plots
using QuadGK
using BlockDiagonals
using PrettyTables

In [2]:
# correlation matrix
function cov_to_cor(covariance)
    std_devs = sqrt.(diag(covariance))
    return covariance ./ (std_devs * std_devs')
end

cov_to_cor (generic function with 1 method)

In [3]:
# fs8 data
function fs8_DESI()

z  = [0.3, 0.51, 0.71, 0.92, 1.32, 1.49]

data  = [0.37784, 0.515897, 0.48384, 0.422208, 0.37468, 0.435]

fs8_upper = [0.4723, 0.572693, 0.539136, 0.470586, 0.418064, 0.48]
fs8_lower = [0.28338, 0.449635, 0.428544, 0.378228, 0.343128, 0.39]
err_up = fs8_upper .- data
err_low = data .- fs8_lower
err_tot = fs8_upper .- fs8_lower ./2

cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err_tot[i]^2
    end
    
    return (data_name = "DESI_fs8", z = z, data = data, cov = cov)
end

fs8_DESI (generic function with 1 method)

In [4]:
fs8data = fs8_DESI();

In [11]:
# DESI data
z1 = [0.3, 0.51, 0.71, 0.92, 1.32, 1.49]
z = repeat(z1, inner = 3)
data_dvrd = [7.788174, 12.514437, 15.675560, 19.676985, 23.861806, 25.708520]
data_dhdm = [3.053800, 1.637266, 1.165867, 0.844996, 0.470709, 0.426508]
data_fs8 = [0.37784, 0.515897, 0.48384, 0.422208, 0.37468, 0.435]
#data_fss8 = [0.377174, 0.513635, 0.483623, 0.422164, 0.376715, 0.434858]

data = 
[7.788174, 3.053800, 0.37784,   # BGS (z=0.30)
12.514437, 1.637266, 0.515897, # LRG1 (z=0.51)
15.675560, 1.165867, 0.48384,  # LRG2 (z=0.71)
19.676985, 0.844996, 0.422208, # LRG3 (z=0.92)
23.861806, 0.470709, 0.37468,  # ELG2 (z=1.32)
25.708520, 0.426508, 0.435  ]   # QSO (z=1.49)

# Full 18x18 Covariance Matrix for DESI 2024 Year 1 
# Variables: [DV/rd, DH/DM, fσs8] per redshift bin
# Source: arXiv:2411.12021 Appendix A
C_total = zeros(18, 18);

# BGS z=0.30
C_total[1:3, 1:3] = 1e-4 * [
    1314.664401 -142.669742  109.427163;
    -142.669742  829.170940 -158.101737;
    109.427163 -158.101737   88.510877
]

# LRG1 z=0.51
C_total[4:6, 4:6] = 1e-4 * [
    541.309833   48.425593   -4.652853;
    48.425593   97.249820  -34.923265;
    -4.652853  -34.923265   41.295470
]

# LRG2 z=0.71
C_total[7:9, 7:9] = 1e-4 * [
    762.457717   39.781004   -1.896006;
    39.781004   36.344098  -17.341350;
    -1.896006  -17.341350   28.119682
]

# LRG3 z=0.92
C_total[10:12, 10:12] = 1e-4 * [
    847.499793   26.038900    3.257324;
    26.038900   16.251088  -10.044074;
    3.257324  -10.044074   22.370314
]

# ELG2 z=1.32
C_total[13:15, 13:15] = 1e-4 * [
2342.506886   26.159601   13.521001;
    26.159601   10.309303   -6.654663;
    13.521001   -6.654663   13.997473
]

# QSO z=1.49
C_total[16:18, 16:18] = 1e-4 * [
3013.788566   -2.205101   36.332110;
    -2.205101    5.845806   -6.747133;
    36.332110   -6.747133   19.785658
]

full_covariance = cov_to_cor(C_total);
#pretty_table(full_covariance)

cov_final = C_total
for k in 1:6
    cov_final[3*k,3*k] = fs8data.cov[k,k]
end

for h in 1:6
    cov_final[(3*h)-2,3*h] = sqrt(cov_final[3*h,3*h] * C_total[(3*h)-2,(3*h)-2]) * full_covariance[(3*h)-2,3*h]
    cov_final[(3*h)-1,3*h] = sqrt(cov_final[3*h,3*h] * C_total[(3*h)-1,(3*h)-1]) * full_covariance[(3*h)-1,3*h]
    cov_final[3*h,(3*h)-2] = sqrt(cov_final[3*h,3*h] * C_total[(3*h)-2,(3*h)-2]) * full_covariance[(3*h)-2,3*h]
    cov_final[3*h,(3*h)-1] = sqrt(cov_final[3*h,3*h] * C_total[(3*h)-1,(3*h)-1]) * full_covariance[(3*h)-1,3*h]
end

    # cov = cov_final
    
    # err_arr = sqrt.(diag(cov))
    # err_dvrd = err_arr[1:3:end]
    # err_dhdm = err_arr[2:3:end]
    # err_fs8 = err_arr[3:3:end]


In [12]:
# hz data
function data_hz()
    z = [
        0.07, 0.09, 0.12, 0.17, 0.179, 0.199, 0.2,
        0.27, 0.28, 0.352, 0.38, 0.3802, 0.4,
        0.4004, 0.4247, 0.44, 0.4497, 0.47, 0.4783,
        0.48, 0.51, 0.593, 0.6, 0.61, 0.68, 0.73,
        0.781, 0.875, 0.88, 0.9, 1.037, 1.3,
        1.363, 1.43, 1.53, 1.75, 1.965
    ]

    data = [
        69.0, 69.0, 68.6, 83.0, 75.0, 75.0, 72.9,
        77.0, 88.8, 83.0, 81.5, 83.0, 95.0, 77.0,
        87.1, 82.6, 92.8, 89.0, 80.9, 97.0, 90.4,
        104.0, 87.9, 97.3, 92.0, 97.3, 105.0,
        125.0, 90.0, 117.0, 154.0, 168.0, 160.0,
        177.0, 140.0, 202.0, 186.5
    ]

    err = [
        19.6, 12.0, 26.2, 8.0, 4.0, 5.0, 29.6,
        14.0, 36.6, 14.0, 1.9, 13.5, 17.0, 10.2,
        11.2, 7.8, 12.9, 23.0, 9.0, 62.0, 1.9,
        13.0, 6.1, 2.1, 8.0, 7.0, 12.0, 17.0,
        40.0, 23.0, 20.0, 17.0, 33.6, 18.0,
        14.0, 40.0, 50.4
    ]

    cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err[i]^2
    end

    return (
        data_name = "data_hz",
        z = z,
        data = data,
        err = err,
        cov = cov
    )
end

hzdata = data_hz();

In [13]:
comb_cov = BlockDiagonal([hzdata.cov, cov_final]);
pretty_table(comb_cov)
min_eigen = minimum(eigvals(comb_cov))
println("Minimum eigenvalue: ", min_eigen)

┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────┬───────────┬───────────┬─────────────┬────────────┬─────────────┬─────────────┬────────────┬─────────────┬────────────┬─────────────┬─────────────┬────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │ Col. 6 │ Col. 7 │ Col. 8 │  Col. 9 │ Col. 10 │ Col. 11 │ Col. 12 │ Col. 13 │ Col. 14 │ Col. 15 │ Col. 16 │ Col. 17 │ Col. 18 │ Col. 19 │ Col. 20 │ Col. 21 │ Col. 22 │ Col. 23 │ Col. 24 │ Col. 25 │ Col. 26 │ Col. 27 │ Col. 28 │ Col. 29 │ Col. 30 │ Col. 31 │ Col. 32 │ Col. 33 │ Col. 34 │ Col. 35 │ Col. 36 │ Col. 37 │   Col. 38 │   Col. 39 │   Col.

In [14]:
# Redshift bins for Version 2 (ShapeFit + BAO)
z2 = [0.3, 0.51, 0.71, 0.92, 1.32, 1.49]

# Updated Data Vectors (Means) from ShapeFit + BAO combination
data_dvrd2 = [7.920703, 12.621670, 15.924595, 19.772506, 24.247096, 25.768648]
data_dhdm2 = [3.014389, 1.574992, 1.181734, 0.825195, 0.486334, 0.425629]
data_fss82 = [0.396326, 0.547632, 0.479541, 0.438454, 0.372661, 0.436849]

# Interleaved Data Vector to match the block-diagonal structure:
# [BGS_params..., LRG1_params..., LRG2_params..., etc.]
data_DESI2 = [
    7.920703, 3.014389, 0.396326,   # BGS (z=0.30)
    12.621670, 1.574992, 0.547632,  # LRG1 (z=0.51)
    15.924595, 1.181734, 0.479541,  # LRG2 (z=0.71)
    19.772506, 0.825195, 0.438454,  # LRG3 (z=0.92)
    24.247096, 0.486334, 0.372661,  # ELG2 (z=1.32)
    25.768648, 0.425629, 0.436849   # QSO (z=1.49)
]

# Full 18x18 Covariance Matrix for Version 2
C_total2 = zeros(18, 18);

# BGS z=0.30
C_total2[1:3, 1:3] = 1e-4 * [
    239.901232  -61.419576   23.441205;
    -61.419576  726.794608 -145.787490;
     23.441205 -145.787490   80.924891
]

# LRG1 z=0.51
C_total2[4:6, 4:6] = 1e-4 * [
    185.794784   -2.289289    9.575015;
     -2.289289   30.722651  -14.614473;
      9.575015  -14.614473   35.284297
]

# LRG2 z=0.71
C_total2[7:9, 7:9] = 1e-4 * [
    310.004892    6.627447    9.878373;
      6.627447   15.261852   -8.194175;
      9.878373   -8.194175   23.061044
]

# LRG3 z=0.92
C_total2[10:12, 10:12] = 1e-4 * [
    300.214114   -2.141102    8.238180;
     -2.141102    5.531413   -4.054349;
      8.238180   -4.054349   17.011310
]

# ELG2 z=1.32
C_total2[13:15, 13:15] = 1e-4 * [
   1205.501318  -15.845487   29.543675;
    -15.845487    4.516231   -3.345970;
     29.543675   -3.345970   11.811326
]

# QSO z=1.49
C_total2[16:18, 16:18] = 1e-4 * [
   2601.484225   -2.427625   40.332929;
     -2.427625    5.679769   -7.100070;
     40.332929   -7.100070   21.028703
]

full_covariance2 = cov_to_cor(C_total2);

cov_final2 = C_total2
for k in 1:6
    cov_final2[3*k,3*k] = fs8data.cov[k,k]
end

for h in 1:6
    cov_final2[(3*h)-2,3*h] = sqrt(cov_final2[3*h,3*h] * C_total2[(3*h)-2,(3*h)-2]) * full_covariance2[(3*h)-2,3*h]
    cov_final2[(3*h)-1,3*h] = sqrt(cov_final2[3*h,3*h] * C_total2[(3*h)-1,(3*h)-1]) * full_covariance2[(3*h)-1,3*h]
    cov_final2[3*h,(3*h)-2] = sqrt(cov_final2[3*h,3*h] * C_total2[(3*h)-2,(3*h)-2]) * full_covariance2[(3*h)-2,3*h]
    cov_final2[3*h,(3*h)-1] = sqrt(cov_final2[3*h,3*h] * C_total2[(3*h)-1,(3*h)-1]) * full_covariance2[(3*h)-1,3*h]
end

In [15]:
comb_cov2 = BlockDiagonal([hzdata.cov, cov_final2]);
pretty_table(comb_cov2)
min_eigen2 = minimum(eigvals(comb_cov2))
println("Minimum eigenvalue: ", min_eigen2)

┌────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────────┬─────────────┬────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬──────────────┬──────────────┬─────────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │ Col. 6 │ Col. 7 │ Col. 8 │  Col. 9 │ Col. 10 │ Col. 11 │ Col. 12 │ Col. 13 │ Col. 14 │ Col. 15 │ Col. 16 │ Col. 17 │ Col. 18 │ Col. 19 │ Col. 20 │ Col. 21 │ Col. 22 │ Col. 23 │ Col. 24 │ Col. 25 │ Col. 26 │ Col. 27 │ Col. 28 │ Col. 29 │ Col. 30 │ Col. 31 │ Col. 32 │ Col. 33 │ Col. 34 │ Col. 35 │ Col. 36 │ Col. 37 │     Col. 38 │    